In [2]:
# Standard libraries
from cellpose import models, core, plot, utils, denoise
from glob import glob

from skimage import exposure, measure

import numpy as np
import matplotlib.pyplot as plt
import cv2

# Custom packages
from util.data.dataset_ht import HtDataset
from util.data.imaging import read_nii
from util.data import imaging  

# Configurations
use_gpu = core.use_gpu()
print(f"GPU activated: {use_gpu}")

from cellpose.io import logger_setup
logger_setup();

GPU activated: False
2024-12-04 13:38:31,349 [INFO] WRITING LOG OUTPUT TO /home/imarcoss/.cellpose/run.log
2024-12-04 13:38:31,350 [INFO] 
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.10.14 
torch version:  	2.3.1


In [2]:
paht = glob(f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/*.tif')

imgs = list(map(imaging.read_image, paht))

In [3]:
[print(np.array(img).shape) for img in imgs]

(1821, 927, 276)
(1161, 705, 305)
(1161, 705, 305)
(1821, 927, 276)


[None, None, None, None]

In [4]:
for i, img in enumerate(imgs):    
    model = denoise.DenoiseModel(
        gpu=use_gpu,
        model_type="denoise_cyto3",
    )
    
    img = np.array(img)
    den = np.swapaxes(np.swapaxes([
        np.array(model.eval(
            img[..., z], channels=[0, 0], diameter=40,
        )) for z in range(img.shape[-1])
    ], 0, 2), 0, 1)
    
    if den.ndim == 4:
        den = den[..., 0]
    
    imaging.save_prediction(den, paht[i].replace('.tif', '_denoised.tif'), 'XYZ', verbose=1)

2024-12-03 17:00:01,607 [INFO] >> denoise_cyto3 << model set to be used
2024-12-03 17:00:01,629 [INFO] ** TORCH CUDA version installed and working. **
2024-12-03 17:00:01,630 [INFO] >>>> using GPU
2024-12-03 17:00:01,845 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


/home/imarcoss/mambaforge/envs/cellpose/lib/python3.11/site-packages/cellpose/resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(file


Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/C4-EZ_denoised.tif
2024-12-03 17:01:57,602 [INFO] >> denoise_cyto3 << model set to be used
2024-12-03 17:01:57,622 [INFO] ** TORCH CUDA version installed and working. **
2024-12-03 17:01:57,624 [INFO] >>>> using GPU
2024-12-03 17:01:57,812 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/C4_MeisDKO_denoised.tif
2024-12-03 17:02:56,867 [INFO] >> denoise_cyto3 << model set to be used
2024-12-03 17:02:56,873 [INFO] ** TORCH CUDA version installed and working. **
2024-12-03 17:02:56,873 [INFO] >>>> using GPU
2024-12-

In [2]:
paht = glob(f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise/*.tif')

imgs = list(map(imaging.read_image, paht))
[print(np.array(img).shape) for img in imgs]

(1821, 927, 276)
(1161, 705, 305)
(1821, 927, 276)
(1161, 705, 305)


[None, None, None, None]

In [3]:
from skimage.restoration import denoise_bilateral


for i, img in enumerate(imgs):        
    img = np.array(img)
    den = np.swapaxes(np.swapaxes([
        denoise_bilateral(
            img[..., z], sigma_color=None, 
            sigma_spatial=25, win_size=7
        ) for z in range(img.shape[-1])
    ], 0, 2), 0, 1)
    
    out_aux = paht[i].split('/')
    name = out_aux[-1].replace('.tif', '_denoised.tif')
    out_aux[-1] = 'CellposeDenoise+Bilateral/' + name
    del out_aux[-2]
    out = '/'.join(out_aux)
    imaging.save_prediction(
        den, out, 
        'XYZ', verbose=1
    )


Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise+Bilateral/C4-EZ_denoised_denoised.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise+Bilateral/C2_MeisDKO_denoised_denoised.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise+Bilateral/C2-EZ_denoised_denoised.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO

In [9]:
paht = glob(f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise/PreProcessing/confocal_3D_unet_sa_meristem_cells/GASP/PostProcessing/C4xC2_MeisDKO_denoised(16)_predictions_gasp_average.tiff')


imgs = list(map(imaging.read_image, paht))
[print(np.array(img).shape) for img in imgs]

(1161, 705, 305)


[None]

In [10]:
for i, img in enumerate(imgs):
    img = np.array(img)
    labeled = measure.label(img)
    sizes = np.bincount(labeled.ravel())
    
    # Exclude bck
    sizes[0] = 0
    
    large_labels = np.where(sizes > 55000)[0]
    large_mask = np.isin(labeled, large_labels)
    
    img[large_mask] = 0
    
    aux = paht[i].split('/')
    name = aux[-1].replace('.tif', '_filtered.tif')
    path = '/'.join(aux[:-1])
    
    imaging.save_prediction(
        img, f'{path}/Filtered/{name}', 
        'XYZ', verbose=1
    )


Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/CellposeDenoise/PreProcessing/confocal_3D_unet_sa_meristem_cells/GASP/PostProcessing/Filtered/C4xC2_MeisDKO_denoised(16)_predictions_gasp_average_filtered.tiff
